# INTELIGENCIA ARTIFICIAL
## Tarea 1: implementacion de gradiente descendiente
### Hecho por: Juan Sebastián Clavijo Martínez<br />
TEMA: Usar el dataset sobre finca raiz en taiwan para hacer una implementación de gradiente descendiente<br />
Fecha: 11-08-2024<br />
Notas: Nada por ahora<br />
**Pontificia Universidad Javeriana**

## 1. preprocesamiento de datos

In [17]:
# Importado de bibliotecas pertinentes
# se usa el #type: ignore para que mi vscode genere warnings sobre la resolucion de dependencias de los pkts

import pandas as pd  # tratamiento de datos en un dataframe
import numpy as np  # numerical python
import seaborn as sns  # biblioteca versatil para estadistica y visualizacion
import matplotlib.pyplot as plt  # biblioteca para vizualizacion de datos
import ipywidgets as widgets  # widgets interactivos, barras deslizantes, etc, para mas info mirar: https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20List.html#file-upload
from IPython.display import display # display interactivo
from ucimlrepo import fetch_ucirepo  # se traen los datos del repositorio, para mas info revisar: https://github.com/uci-ml-repo/ucimlrepo (el dataset tiene el id=447)

In [18]:
# Parte 1.1: carga de datos
# se trae el dataset
url = 'https://archive.ics.uci.edu/static/public/477/data.csv'
# se hace el obj dataframe
df_rev = pd.read_csv(url) ##Se llama así por Real Estate Valuation
# se presentan los primeros 5 renglones y la forma de los datos
df_rev.head(5)

,No,X1 transaction date,X2 house age,X3 distance to the nearest MRT station,X4 number of convenience stores,X5 latitude,X6 longitude,Y house price of unit area
0,1,2012.917,32.0,84.87882,10,24.98298,121.54024,37.9
1,2,2012.917,19.5,306.59470,9,24.98034,121.53951,42.2
2,3,2013.583,13.3,561.98450,5,24.98746,121.54391,47.3
3,4,2013.500,13.3,561.98450,5,24.98746,121.54391,54.8
4,5,2012.833,5.0,390.56840,5,24.97937,121.54245,43.1


In [19]:
# Parte 1.2: limpieza y preparación de los datos (cuenta de datos null, etc. y limpieza)
desaparecidos = len(df_rev) - len(df_rev.dropna())
Cantidad = len(df_rev)
print("Cantidad de datos observados con datos NaN", desaparecidos)
print("Cantidad de datos duplicados", df_rev.duplicated().sum())
##se eliminan los datos Null y Duplicados
df_rev.dropna(inplace=True)
df_rev.drop_duplicates(inplace=True)
# Se reinicia el indice por los elementos eliminados
df_rev.reset_index(drop=True, inplace=True)

Cantidad de datos observados con datos NaN 0
Cantidad de datos duplicados 0


## 2. Implementación de Gradiente Descendiente

In [20]:
# Parte 2.1 Gráfico interactivo con los datos sin implementar GD
df_rev_01 = df_rev.rename(
    columns={
        "X1 transaction date": "transaction date",
        "X2 house age": "house age",
        "X3 distance to the nearest MRT station": "distance to the nearest MRT station",
        "X4 number of convenience stores": "number of convenience stores",
        "X5 latitude": "latitude",
        "X6 longitude": "longitude",
    }
)

# Eliminamos columnas que no serán usadas en el eje X
x_columns = df_rev_01.columns.drop(["Y house price of unit area", "No"])

# Función para graficar
def plot_graph(x_column):
    x = df_rev_01[x_column]
    y = df_rev_01["Y house price of unit area"]

    fig, ax = plt.subplots()
    ax.scatter(x, y, alpha=0.4, edgecolor="none")
    ax.set_xlabel(x_column)
    ax.set_ylabel(
        "house price of unit area [10k TWD/Ping(1 Ping = 3.3 meter^2)]"
    )
    ax.set_title("Gráfica interactiva")
    plt.show()

# Creamos un menú desplegable para seleccionar la columna del eje X
x_dropdown = widgets.Dropdown(
    options=x_columns, value="house age", description="Eje X:"
)

# Mostramos el widget y la gráfica
widgets.interactive(plot_graph, x_column=x_dropdown)

interactive(children=(Dropdown(description='Eje X:', index=1, options=('transaction date', 'house age', 'dista…

### Flujo de GD
1. proponer el modelo lineal (la matriz A refleja la estruc del modelo)
2. definir un theta inicial aleatoriamente
3. estimar salida del modelo para ese theta h=A*theta
4. encontrar el error
5. actualizar el theta = theta - (alpha/m(numero de datos))xA^t*e
6. $y=mx+b$
7. goto 3 until convergence
    - crieterios de parada:
      - ubmral de error
      - no hay cambio en theta
- util: no actualizar h! solo se vuelve a calcular cuando se actualizan los parametros

In [21]:
# Renombrar la columna 'Y house price of unit area'
df_rev_01.rename(
    columns={"Y house price of unit area": "house price of unit area"}, inplace=True
)

# Eliminar la columna 'No'
df_rev_01.drop(columns=["No"], inplace=True)

# Agregar una columna de 1s para el término de sesgo
df_rev_01["ones"] = 1

# Extraer las características (X) y la variable objetivo (y)
X = df_rev_01.drop(columns=["house price of unit area"]).values
y = df_rev_01["house price of unit area"].values


# Función para calcular h = X * theta
def calculo_h(X, theta):
    return X @ theta


# Función para calcular el error h - y
def calculo_error(h, y):
    return h - y


# Función para calcular la derivada de la función de costo
def funcion_de_costo(X, e):
    m = len(e)
    return (1 / m) * (X.T @ e)


# Implementación del Gradiente Descendente
def gradiente_descendiente(X, y, alpha=0.01, iteraciones=1000):
    m, n = X.shape
    theta = np.random.randn(n)

    for i in range(iteraciones):
        h = calculo_h(X, theta)
        e = calculo_error(h, y)
        gradiente = funcion_de_costo(X, e)

        # Actualizar theta
        theta = theta - alpha * gradiente

        # Imprimir el costo en cada iteración
        if i % 100 == 0:
            costo = (1 / (2 * m)) * np.sum(e**2)
            print(f"Iteración {i+1}, Costo: {costo:.4f}")

    return theta


# Ejecutar el Gradiente Descendente y obtener los parámetros theta
theta_final = gradiente_descendiente(X, y)

print("Theta final:", theta_final)

Iteración 1, Costo: 5811328.3076
Iteración 101, Costo: nan
Iteración 201, Costo: nan
Iteración 301, Costo: nan
Iteración 401, Costo: nan
Iteración 501, Costo: nan
Iteración 601, Costo: nan
Iteración 701, Costo: nan
Iteración 801, Costo: nan
Iteración 901, Costo: nan
Theta final: [nan nan nan nan nan nan nan]


C:\Users\juanc\AppData\Local\Temp\ipykernel_17760\2690531948.py:30: RuntimeWarning: overflow encountered in matmul
  return (1 / m) * (X.T @ e)
C:\Users\juanc\AppData\Local\Temp\ipykernel_17760\2690531948.py:30: RuntimeWarning: invalid value encountered in matmul
  return (1 / m) * (X.T @ e)
C:\Users\juanc\AppData\Local\Temp\ipykernel_17760\2690531948.py:44: RuntimeWarning: invalid value encountered in subtract
  theta = theta - alpha * gradiente


In [22]:
# Parte 2.2 Implementacion de GD
# ya que en df_rev_01 todavía hay columnas sobrantes y mal nombradas, se actualizan para mayor conveniencia
# Renombrar la columna
df_rev_01.rename(
    columns={"Y house price of unit area": "house price of unit area"}, inplace=True
)

# Eliminar la columna 'No'
df_rev_01.drop(columns=["No"], inplace=True)

# agregar a la matriz de los datos una columna con 1s
df_rev_01["ones"] = 1

# se hace el calculo de h
def calculo_h(theta):
    h = df_rev_01*theta
    return h

# funcion de calculado de error: 
# y son los valores de la columna de los y, para esta ejemplo house price of unit area
def calculo_error(h, y):
    e = h-y
    return e
    
# sacar la derivada de la func. de costo (es igual a x^T*error(error= h-y)) mirar video
def funcion_de_costo():
    print("funciondecosto!")
    
# ESTE ES EL ULTIMO PASO actualizar theta (for yo defino el numero de vedces que se repite)
# aqui hago el for
def gradiente_descendiente():
    # la dim del vec theta es la dim de la matriz incl la col de 1s
    theta = np.random.randn(df_rev_01.shape[1])
    # Definimos la tasa de aprendizaje y el número de iteraciones
    alpha = 0.01
    iteraciones = 1000
    y = df_rev_01['house price of unit area']
    m = df_rev_01.shape[0]
    for i in range(1):
        h = calculo_h(theta)
        e = calculo_error(h, y)
        #theta = theta-((alpha/m)*df_rev_01.transpose*e)
        
gradiente_descendiente() #imprimir valores del costo
# ACUMULAR LOS VALS DE THETA PARA GRAFICAR Y VER EVOLIUCION - grafica despues de la tarea del notebook de la clase

KeyError: "['No'] not found in axis"